## Import Libraries

**Running a simple 3 layer CNN after converting data to 10x10 matrix**

In [ ]:
!pip -q install attrdict

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
from glob import glob
from itertools import chain
import cv2
import random
from tqdm.notebook import tqdm
import time

In [ ]:
from attrdict import AttrDict
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense
from tensorflow.keras.layers import Flatten, Input, Layer, MaxPooling2D
from tensorflow.keras.models import Model

## Load Data

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")
sample_submission = pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv")

In [ ]:
train = train.drop(columns=["id"])
test = test.drop(columns=["id"])

In [ ]:
print(f"Train Shape :  {train.shape}")
print(f"Test Shape :  {test.shape}")

## Configurations

In [ ]:
config = dict(SEED = 42,
              IMG_HEIGHT = 10,
              IMG_WIDTH = 10,
              EPOCHS = 100,
              BATCH_SIZE = 64,
              LR = 0.01)
config = AttrDict(config)

## Utility Functions

In [ ]:
def seed_everything(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
def plot_hist(hist):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].plot(hist.history["loss"])
    ax[0].plot(hist.history["val_loss"])
    ax[0].set_title("Loss Plot")
    ax[0].set_ylabel("Loss")
    ax[0].set_xlabel("Epoch")
    ax[0].legend(["Train", "Validation"], loc="upper left")

    ax[1].plot(hist.history["root_mean_squared_error"])
    ax[1].plot(hist.history["val_root_mean_squared_error"])
    ax[1].set_title("RMSE Plot")
    ax[1].set_ylabel("RMSE")
    ax[1].set_xlabel("Epoch")
    ax[1].legend(["Train", "Validation"], loc="upper left")
    plt.show()

In [ ]:
def plot_metrics(hist):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].plot(hist.history["mean_absolute_error"])
    ax[0].plot(hist.history["val_mean_absolute_error"])
    ax[0].set_title("MAE Plot")
    ax[0].set_ylabel("MAE")
    ax[0].set_xlabel("Epoch")
    ax[0].legend(["Train", "Validation"], loc="upper left")

    ax[1].plot(hist.history["mean_squared_error"])
    ax[1].plot(hist.history["val_mean_squared_error"])
    ax[1].set_title("MSE Plot")
    ax[1].set_ylabel("MSE")
    ax[1].set_xlabel("Epoch")
    ax[1].legend(["Train", "Validation"], loc="upper left")

In [ ]:
def plot_grid(dataset, h=3, w=3, title=""):
    f, ax = plt.subplots(h, w, figsize=(30, 30))
    for images, labels in dataset.shuffle(100).take(1):
        for i in range(h*w):
            img = (images[i] * 255).numpy().astype("uint8")
            ax[i // h, i % w].imshow(img[:, :, 0])
            ax[i // h, i % w].axis("off")
            ax[i // h, i % w].set_title(labels[i].numpy(), fontdict={"fontsize": 20})
    plt.tight_layout()
    f.suptitle(title, fontsize="large", fontweight="extra bold")
    plt.show()

In [ ]:
def print_metrics(stage, y_true, y_pred):
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    rmse = metrics.mean_squared_error(y_true, y_pred, squared=False)
    print(f"{stage} Mean Absolute Error: {mae:.2f}")
    print(f"{stage} Mean Squared Error: {mse:.2f}")
    print(f"{stage} Root Mean Squared Error: {rmse:.2f}")

## Train-Validation Split

In [ ]:
X = train[test.columns.tolist()]
y = train["loss"]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y)

Here the validation split is not representative of test data to be honest

In [ ]:
print(f"Train Shape: {X_train.shape, y_train.shape}")
print(f"Validation Shape: {X_valid.shape, y_valid.shape}")

## Normalizing or Standard Scaling

In [ ]:
total = pd.concat([train[test.columns.tolist()], test])
total.shape

While normalization brings values to 0-1 range, it doesn't keep the shape the same.

On the other hand standard scaling keeps shape the same but subtracts mean & divides by standard deviation such that mean is 0 & standard deviation is 1

In [ ]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(total)

In [ ]:
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(test)

**Reshaping to `10x10x1` matrix**

In [ ]:
X_train = X_train.reshape(-1, 10, 10, 1)
X_valid = X_valid.reshape(-1, 10, 10, 1)
X_test = X_test.reshape(-1, 10, 10, 1)

In [ ]:
X_train.shape, X_valid.shape, X_test.shape

## Convert to TensorFlow Dataset

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.batch(config.BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
plot_grid(train_dataset, title="Training Images")

In [ ]:
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
valid_dataset = valid_dataset.batch(config.BATCH_SIZE)
valid_dataset = valid_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
plot_grid(valid_dataset, title="Validation Images")

## Tensorflow doesn't provide RMSE loss function

In [ ]:
class RootMeanSquaredError(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        y_pred = tf.convert_to_tensor(y_pred)
        y_true = tf.cast(y_true, y_pred.dtype)
        return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true), axis=-1))

## Simple CNN - Regression
Refer my original solution [here](https://www.kaggle.com/aditya08/count-the-boxes-cnn-regression-tensorflow) for reference

In [ ]:
def build_model():
    # Inputs to the model
    input_img = Input(
        shape=(config.IMG_HEIGHT, config.IMG_WIDTH, 1), name="image", dtype="float32"
    )

    x = Conv2D(8, (2, 2), activation="relu", padding="same", name="conv_1")(input_img)
    x = Conv2D(16, (2, 2), activation="relu", padding="same", name="conv_2")(x)
    x = MaxPooling2D((2, 2), name="pool_1")(x)
    x = Conv2D(32, (2, 2), activation="relu", padding="same", name="conv_3")(x)
    x = Conv2D(64, (2, 2), activation="relu", padding="same", name="conv_4")(x)
    x = MaxPooling2D((2, 2), name="pool_2")(x)
    x = Flatten()(x)
    x = Dense(256, activation="relu", name="dense_1")(x)
    x = Dense(128, activation="relu", name="dense_2")(x)
    x = Dense(64, activation="relu", name="dense_3")(x)
    x = Dense(16, activation="relu", name="dense_4")(x)
    output = Dense(1, activation="linear", name="output")(x)

    model = Model(inputs=[input_img], outputs=output, name="regression_model")

    opt = tf.keras.optimizers.Adam(learning_rate=config.LR)

    model.compile(optimizer=opt, 
                  loss=tf.keras.losses.MeanSquaredError(), 
                  metrics=[tf.keras.metrics.RootMeanSquaredError(), 
                           tf.keras.metrics.MeanAbsoluteError(), 
                           tf.keras.metrics.MeanSquaredError()])
    return model

## Training

In [ ]:
model = build_model()
model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
early_stopping_patience = 10
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=early_stopping_patience, 
    restore_best_weights=True
)

checkpoint_filepath = "model_checkpoint"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor="val_loss",
    mode="min",
    save_best_only=True)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=config.EPOCHS,
    callbacks=[early_stopping, model_checkpoint_callback]
)

> Somehow the loss value & rmse metric are different - ideally they should be the same. Need checking!

In [ ]:
plot_hist(history)

In [ ]:
plot_metrics(history)

## Inference
Always load the best model

In [ ]:
best_model = tf.keras.models.load_model(checkpoint_filepath, compile=False)

**Validation Data Evaluation**

In [ ]:
valid_preds = best_model.predict(valid_dataset)
valid_preds = list(chain.from_iterable(valid_preds))

In [ ]:
print_metrics("Validation", y_valid, valid_preds)

**Training Data Evaluation**

In [ ]:
train_preds = best_model.predict(train_dataset)

In [ ]:
train_preds = list(chain.from_iterable(train_preds))

In [ ]:
print_metrics("Training", y_train, train_preds)

## Submission

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices(X_test)
test_dataset = test_dataset.batch(config.BATCH_SIZE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
test_preds = best_model.predict(test_dataset)

In [ ]:
test_preds = list(chain.from_iterable(test_preds))

In [ ]:
sample_submission["loss"] = test_preds

In [ ]:
sample_submission["loss"].describe()

In [ ]:
sample_submission["loss"].quantile(np.linspace(.1, 1, 9, 0))

In [ ]:
sample_submission.to_csv("submission.csv", index=False)